## ETL02_Company_report

In [1]:
from pyspark.sql.types import *
from pyspark.sql import functions as f
import requests
import pandas as pd 
from notebookutils import mssparkutils

In [36]:
# get data from api 
def get_financial_statement(symbol,limit,apikey):
    url ='https://financialmodelingprep.com/api/v3/financial-statement-full-as-reported/{0}?period=annual&limit={1}&apikey={2}'.format(symbol,limit,apikey)
    r = requests.get(url)
    data = r.json()
    return data
# turn pandas df into spark datafrme and change scheme
def read_into_sp(dataframe):
    result = (spark.createDataFrame(dataframe))
    return result

In [61]:
if __name__ == "__main__":
    
    # apikey = mssparkutils.credentials.getSecret('synapse-key1234','fmp-key')
    # # set up target ticker 
    # symbol_list=['NVDA','AMD','INTC','QCOM','GOOG','MSFT','AMZN','AAPL']
    # # get data from api
    # stock_data = [get_financial_statement(symbol,5,apikey) for symbol in symbol_list]
    list_ = [pd.DataFrame(data) for data in stock_data]
    selected_columns=[
        "date","symbol","period","documenttype","revenues","costofrevenue",
        "grossprofit","netincomeloss","earningspersharebasic","earningspersharediluted","operatingincomeloss",
        "operatingexpenses","researchanddevelopmentexpense","sellinggeneralandadministrativeexpense",
        "interestexpense","cashandcashequivalentsatcarryingvalue","commonstocksharesoutstanding","longtermdebt"
    ]
    pdfs = (pd.concat(list_ ))[[*selected_columns]]
    final_df_sp = read_into_sp(pdfs)

In [63]:
#add year 
datalake_nm = 'datalake'+mssparkutils.env.getWorkspaceName()[7:] # get datalake name
file_path ='abfss://files@{0}.dfs.core.windows.net/synapse/workspaces/data/company_report'.format(datalake_nm)
final_df_sp.write.option("header", True).csv(file_path,mode='overwrite')